In [1]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Step 1: Load the Iris dataset
iris = load_iris()
X = iris.data  # Feature matrix
y = iris.target  # Target vector

# Step 2: Preprocess the data - binary classification (Setosa vs non-Setosa)
# Convert to binary classification: Setosa (original class 0) vs others
y_binary = np.where(y == 0, 1, 0)  # Setosa=1, others=0

# Step 3: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y_binary, test_size=0.2, random_state=42
)

# Add a column of ones to X for the bias term (intercept)
X_train = np.c_[np.ones(X_train.shape[0]), X_train]
X_test = np.c_[np.ones(X_test.shape[0]), X_test]

# Step 4: Define the cost function (logistic loss)
def logistic_loss(y_true, y_pred):
    # Clip predictions to avoid log(0)
    y_pred = np.clip(y_pred, 1e-15, 1 - 1e-15)
    return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

# Sigmoid function
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# Step 5: Define the training function
def train_logistic_regression(X, y, learning_rate=0.01, n_iterations=1000, lambda_=0.1):
    n_samples, n_features = X.shape
    # Initialize weights (including bias)
    W = np.zeros(n_features)
    
    for _ in range(n_iterations):
        # Calculate predictions
        linear_model = np.dot(X, W)
        y_pred = sigmoid(linear_model)
        
        # Calculate gradients
        dw = (1 / n_samples) * np.dot(X.T, (y_pred - y)) + (lambda_ / n_samples) * W
        # Update weights
        W -= learning_rate * dw
        
        # Optional: Print loss every 100 iterations
        if _ % 100 == 0:
            loss = logistic_loss(y, y_pred)
            print(f"Iteration {_}, Loss: {loss}")
    
    return W

# Step 6: Train the model
W = train_logistic_regression(
    X_train, y_train, 
    learning_rate=0.1, 
    n_iterations=1000, 
    lambda_=0.1
)

# Extract bias and weights
b = W[0]
weights = W[1:]

# Step 7: Define the prediction function
def predict(X, W):
    linear_model = np.dot(X, W)
    y_pred = sigmoid(linear_model)
    return (y_pred >= 0.5).astype(int)

# Step 8: Predict on the test set
y_pred = predict(X_test, W)

# Step 9: Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("\nModel Evaluation:")
print(f"Accuracy: {accuracy:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)

Iteration 0, Loss: 0.6931471805599453
Iteration 100, Loss: 0.05820894526634577
Iteration 200, Loss: 0.031783648804123744
Iteration 300, Loss: 0.022409914220723907
Iteration 400, Loss: 0.01757340100172084
Iteration 500, Loss: 0.014611598880809422
Iteration 600, Loss: 0.012608230534742507
Iteration 700, Loss: 0.011161908545838135
Iteration 800, Loss: 0.010068539330948667
Iteration 900, Loss: 0.009213178341904604

Model Evaluation:
Accuracy: 1.0000

Confusion Matrix:
[[20  0]
 [ 0 10]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        20
           1       1.00      1.00      1.00        10

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30

